CODE MODELS

FINAL HYBRID APPROACH FOR UI INTEGRATION

In [ ]:

from werkzeug.utils import secure_filename
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, VotingRegressor
from sklearn.metrics import r2_score, mean_squared_error
import xgboost as xgb
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import time

# ? Hybrid
# ! HYBRID MODEL
# ? Hybrid

hybrid_start_time = time.time()
imputer = SimpleImputer(strategy='mean')
data_imputed = pd.DataFrame(imputer.fit_transform(data), columns=data.columns)

data_imputed['Volume'] = data_imputed['Surface Area'] * data_imputed['Overall Height']
data_imputed['Wall/Roof Ratio'] = data_imputed['Wall Area'] / data_imputed['Roof Area']

data_final = data_imputed

X = data_final.drop(['Heating Load', 'Cooling Load'], axis=1)
Y1 = data_final['Heating Load']
Y2 = data_final['Cooling Load']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, Y1_train, Y1_test, Y2_train, Y2_test = train_test_split(X_scaled, Y1, Y2, test_size=0.2, random_state=20)

rf_model_heating = RandomForestRegressor(random_state=20, n_estimators=500, max_depth=70, min_samples_split=3, min_samples_leaf=1, max_features='sqrt', bootstrap=False)
rf_model_heating.fit(X_train, Y1_train)

rf_model_cooling = RandomForestRegressor(random_state=20, n_estimators=500, max_depth=70, min_samples_split=3, min_samples_leaf=1, max_features='sqrt', bootstrap=False)
rf_model_cooling.fit(X_train, Y2_train)

xgb_model_heating = xgb.XGBRegressor(objective='reg:squarederror', eval_metric='rmse', subsample=0.8, random_state=20, n_estimators=2000, max_depth=70, learning_rate=0.05, min_child_weight=2, gamma=0, colsample_bytree=0.6)
xgb_model_heating.fit(X_train, Y1_train)

xgb_model_cooling = xgb.XGBRegressor(objective='reg:squarederror', eval_metric='rmse', subsample=0.8, random_state=20, n_estimators=2000, max_depth=70, learning_rate=0.05, min_child_weight=2, gamma=0, colsample_bytree=0.6)
xgb_model_cooling.fit(X_train, Y2_train)

heating_models = [('Random Forest', rf_model_heating), ('XGBoost', xgb_model_heating)]
heating_ensemble = VotingRegressor(heating_models)
heating_ensemble.fit(X_train, Y1_train)

cooling_models = [('Random Forest', rf_model_cooling), ('XGBoost', xgb_model_cooling)]
cooling_ensemble = VotingRegressor(cooling_models)
cooling_ensemble.fit(X_train, Y2_train)

Y1_pred_train_ensemble = heating_ensemble.predict(X_train)
Y1_pred_test_ensemble = heating_ensemble.predict(X_test)

Y2_pred_train_ensemble = cooling_ensemble.predict(X_train)
Y2_pred_test_ensemble = cooling_ensemble.predict(X_test)

# Evaluation metrics for Heating Load
r2_train_ensemble_heating = r2_score(Y1_train, Y1_pred_train_ensemble)
r2_test_ensemble_heating = r2_score(Y1_test, Y1_pred_test_ensemble)
mse_train_ensemble_heating = mean_squared_error(Y1_train, Y1_pred_train_ensemble)
mse_test_ensemble_heating = mean_squared_error(Y1_test, Y1_pred_test_ensemble)
rmse_train_ensemble_heating = np.sqrt(mse_train_ensemble_heating)
rmse_test_ensemble_heating = np.sqrt(mse_test_ensemble_heating)

# Evaluation metrics for Cooling Load
r2_train_ensemble_cooling = r2_score(Y2_train, Y2_pred_train_ensemble)
r2_test_ensemble_cooling = r2_score(Y2_test, Y2_pred_test_ensemble)
mse_train_ensemble_cooling = mean_squared_error(Y2_train, Y2_pred_train_ensemble)
mse_test_ensemble_cooling = mean_squared_error(Y2_test, Y2_pred_test_ensemble)
rmse_train_ensemble_cooling = np.sqrt(mse_train_ensemble_cooling)
rmse_test_ensemble_cooling = np.sqrt(mse_test_ensemble_cooling)


# ! HEATING
hybrid_heating_data = list(zip(Y1_train.tolist(), Y1_pred_train_ensemble.tolist()))
# ? COOLING
hybrid_cooling_data = list(zip(Y2_train.tolist(), Y2_pred_train_ensemble.tolist()))

# ! HEATING TEST
hybrid_heating_data_test = list(zip(Y1_test.tolist(), Y1_pred_test_ensemble.tolist()))
# ? COOLING TEST
hybrid_cooling_data_test = list(zip(Y2_test.tolist(), Y2_pred_test_ensemble.tolist()))


# ! Combining training and testing Metrics

# ? R-Squared for TRAINING
weight_heating = 2
weight_cooling = 1
harmonic_r2_mean_training = (r2_train_ensemble_heating * weight_heating + r2_train_ensemble_cooling * weight_cooling) / (weight_cooling + weight_heating)
# harmonic_r2_mean_training = 2 / ((1 / r2_train_ensemble_heating) + (1 / r2_train_ensemble_cooling))
# ? R-Squared for TESTING
weight_heating = 2
weight_cooling = 1
harmonic_r2_mean_testing = (r2_test_ensemble_heating * weight_heating + r2_test_ensemble_cooling * weight_cooling) / (weight_cooling + weight_heating)
# harmonic_r2_mean_testing = 2 / ((1 / r2_test_ensemble_heating) + (1 / r2_test_ensemble_cooling))

# ? MSE for TRAINING
weight_heating = 2
weight_cooling = 1
weighted_average_mse_training = (mse_train_ensemble_heating * weight_heating + mse_train_ensemble_cooling * weight_cooling) / (weight_cooling + weight_heating)

# ? MSE for Testing
weight_heating = 2
weight_cooling = 1
weighted_average_mse_testing = (mse_test_ensemble_heating * weight_heating + mse_test_ensemble_cooling * weight_cooling) / (weight_cooling + weight_heating)

# ? RMSE for Testing
weight_heating = 2
weight_cooling = 1
weighted_average_rmse_training = (rmse_train_ensemble_heating * weight_heating + rmse_train_ensemble_cooling * weight_cooling) / (weight_cooling + weight_heating)

# ? RMSE for Testing
weight_heating = 2
weight_cooling = 1
weighted_average_rmse_testing = (rmse_test_ensemble_heating * weight_heating + rmse_test_ensemble_cooling * weight_cooling) / (weight_cooling + weight_heating)

hybrid_elapsed_time = time.time() - hybrid_start_time

total_elapsed_time = time.time() - start_time